# 8章 ディープラーニング

## 8.1 ネットワークをより深く

### 8.1.1 よりディープなネットワークへ
手書き文字の認識精度を上げるため、CNNをより深くするためVGGと言うネットワークを参考に構築してみる。

「Conv-ReLU-Conv-ReLU-Pool - Conv-ReLU-Conv-ReLU-Pool - Conv-ReLU-Conv-ReLU-Pool - Affine-ReLU-Dropout - Affine-Dropout-Softmax」

このネットワークは層が深くなるに連れてチャンネル数が大きくなるのが特徴である。(畳み込み層のチャンネルが前層から順に16,16,32,32,64,64となる)他の特徴としては以下が挙げられる。

* 3×3の小さなフィルターによる畳み込み層
* 活性化関数はReLU
* 全結合相の後にDropoutレイヤを試用
* Adamによる最適化
* 重みの初期値として「Heの初期値」を使用

### 8.1.2 更に認識精度を高めるには
MNISTのデータセット程度であれば、さほどディープでなくても精度が出る(畳み込み層が2層、全結合層が2層程度)。

他に精度を高めるための手法として、アンサンブル学習や学習係数の減衰(learning rate decay)、Data Augmentation(データ拡張)などがある。

Data Augmentationは入力データを人工的に拡張することである。入力画像に対して回転や縦横方向の移動などの微小な変化を与えて画像枚数を増やす。データセットが限られているときに特に有効である。
他にも画像の中から一部を切り出す「crop処理」や左右をひっくり返す「flip処理」などである。一般的な画像であれば輝度などの見た目の変化や拡大縮小なども有効である。

### 8.1.3 層を深くすることのモチベーション
層を深くする事について理論的にはまだまだはっきりしないが、研究や実験により説明出来ることが出てきている。

ILSVRCに代表される大規模画像認識のコンペにおいてネットワーク層を深くする手法が好成績を修めている。これは層を深くすることに比例し、認識性能が向上していると読み取れる。

層を深くすることの利点としてネットワークのパラメータ数の削減がある。5×5のフィルターからなる畳み込みそうではパラメータが25個必要だが、3×3のフィルターを利用する2層のネットワークであると2×3×3=18個のパラメータを用意すれば同等の表現を行なうことができる。これは層を深くすればするほど差が顕著となる。

つまりパラメータを小さくしつつ、受容野(receptive field)を広くカバー出来ることである。更に層を重ねることでReLUなどの活性化関数が挟まれ表現力が増す。これはネットワークに「非線形」の力が加わるためであう。

学習の効率性も層を深くする利点である。層を深くすることで学習データを少なく出来、高速に学習が行える。これは層に応じて学習する特徴を分別することができるためである(犬をいきなり学ぶのでなくエッジやブロブ
認識すれ層、テクスチャを認識する層など分担できる)。

また、層を深くすることで階層的に情報を渡すことができる。エッジの次にテクスチャなど高度なパターンを効率よく学習することを期待できる。解きやすいシンプルな問題へと分解できる。



## 8.2 ディープラーニングの小歴史

### 8.2.1 ImageNet
100万枚を超える様々な画像のデータセット。このデータセットを使ってILSVRCという画像認識のコンペティションが毎年行われている。

クラス分類のタスクにおいては2015年にReｓNetが誤認識率3.5%になっている。画像認識においてVGG,GoogLeNet,ResNetは有名なネットワークである。

### 8.2.3 VGG
畳み込みそうとプーリング層から構成される基本的なCNN。重みのある層(畳み込み層や全結合層)を全部で16層(もしくは19層)重ねている点が特徴。層の深さに応じて「VGG16」や「VGG19」と呼ばれる。

VGGは3×3の小さなフィルターによる畳み込み層を連続して行っている。畳み込み層を2回から4回連続しプーリング層でサイズを半分にするという処理を繰り返し行なう。最後に全結合相を経由して出力する。

近年ではGoogLeNetに負けてしまったが、比較的シンプルなので好んで用いられる。


### 8.2.4 GoogLeNet

これまで見てきたCNNと同じ構成だが、ネットワークが縦方向の深さだけでなく、横方向にも深さを持つ。横方向の幅は「インセプション構造」と呼ばれている。

インセプション構造ではサイズの異なるフィルターとプーリングを複数適用し結果を結合する。この構造をひとつのビルディングブロック(構成要素)として用いるのがGoogLeNetの特徴である。また、サイズが1×1のフィルターの畳み込み層多くの場所で用いる。1×1の畳み込み演算はチャンネル方向にサイズを減らすことでパラメータ削減や処理の高速化に貢献する。


### 8.2.5 ResNet
Microsoftによって開発された。
ディープラーニングにおいて層を深くしすぎると学習がうまくいかず性能が劣ることがある。このネットワークでは「スキップ構造」(「ショートカット」「バイパス」とも)を用いて解決する。スキップ構造とは入力データを畳み込み層をまたいで出力に合算する構造を言う。

例としては2連続する畳み込み層をスキップた入力データを、畳み込み層の出力に加算する。逆伝播の際にスキップ構造によって信号が減衰することなく伝えることにもなる。これにより勾配消失問題が軽減されることが期待できる。


ImageNetの巨大なデータセットを使って学習した重みデータを有効活用することを「転移学習」といい、学習済みの重みを別のニューラルネットワークにコピーして再学習を行なう事がある。例えばVGGと同じネットワークを用意し、学習済みの重みを初期値とし、新しいデータセットを対象に再学習(fine tuning)を行なう。手元にあるデータセットが少ない場合に有効である。



## 8.3 ディープラーニングの高速化

### 8.3.1 取り組むべき問題
ディープラーニングに於いてどの処理に時間が費やされているか確認すると、AlexNetにおいては畳み込み層においてGPU利用で95%、CPUで89%が使われている。

### 8.3.2 GPUによる高速化
ディープラーニングでは大量の積和演算を行う必要であり、GPUが得意とする分野である。よってim2colなどで巨大な行列にしてしまい、大きなまとまりで計算させるのはGPUと相性が良い。

### 8.3.3 分散学習
GPUを用いても大量に時間がかかってしまうため、分散学習という手法がある。TensorflowやCNTKでは分散学習を用いることができる。

### 8.3.4 演算精度のビット削減
計算量に加え、メモリ容量やバス帯域などがボトルネックになる。大量の重みパラメータや中間データはメモリを大量に消費する。バス帯域ではGPUやCPUのバスを流れるデータが増加して制限を超えるとネックになる。

数値精度に関してディープラーニングわかっていることは、そこまで精度を必要としない事が分かっている。これはニューラルネットワークがロバスト性を備えるためであり、データを劣化させてノイズを含ませても出力結果への影響は小さい。

ディープラーニングにおいては16ビットの半精度浮動小数点数でも問題なく学習できることが分かっている。(サンプルch08/harf_float_network.py)
最新の研究では重みや中間データを1ビットで表現する「Binarized Neural Networkd」が提案されている。


## 8.4 ディープラーニングの実用例

### 8.4.1 物体検出
画像中から物体のいちの特定を含めてクラス分類を行なう問題。画像全体を対象とする物体認識より困難。

CNNを用いて文体検出を行う手法としてR-CNNがある。これは候補領域抽出とCNN特徴の計算の2パートに分かれる。最初に何らかの方法でオブジェクト領域を探し出し、その領域についてCNNを適用して分類する。

R-CNNの論文では候補領域抽出にコンピュータビジョンの手法であるSelective Searchが使われている。最近では候補領域抽出にもCNNを用いるFaster R-CNNが提案されている。

### 8.4.2 セグメンテーション
画像に対してピクセルレベルでクラス分類を行なう問題。教師データとしてはピクセル単位でオブジェクトごとに色付けを行ない、学習させる必要がある。推論の際には入力画像全てのピクセルに対してクラス分類を行う。

ただし、それだとピクセル数だけforward処理を行なう必要があるため多くの時間が必要となってしまう。そこでFCN(Fully Convolutional Network)という手法が提案され、一回のforward処理で全てのピクセルに対してクラス分類を行なう。この手法は全結合層を同じ働きをする畳み込み層に置き換えることによって解決する。
### 8.4.3 画像キャプション生成
画像のみから説明する文書を自動で生成する研究。
代表的な手法にNIC(Neural Image Caption)と呼ばれるモデルがある。NICはディープなCNNと自然言語を扱うためのRNN(Recurrent Neural Network)から構成される。
画像からCNNによって特徴を抽出し、RNNに渡す。RNNではCNNが抽出した特徴を初期値としてテキストを再帰的に生成する。

このような画像と自然言語といった複数の種類の情報を組み合わせて処理することを「マルチモーダル処理」という。




## 8.5 ディープラーニングの未来

